Jake Gluck - Capital News Service - jagluck.github.io

NBA Twitter Analysis

In [2]:
#import libraries
import pandas as pd
import numpy as np
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [26]:
#load twitter conv data gathered by the scraper
df = pd.read_csv('/Users/jagluck/Documents/GitHub/nba-twitter/nba-conv.csv')

NBACAPS = {'CAVS','OKCTHUNDER','CELTICS','NYKNICKS','BROOKLYNNETS','PELICANSNBA', 'PACERS', 'ORLANDOMAGIC','TIMBERWOLVES','MIAMIHEAT',
'HORNETS', 'DETROITPISTONS', 'DALLASMAVS', 'LACLIPPERS', 'LAKERS', 'UTAHJAZZ', 'NUGGETS', 'WASHWIZARDS', 'CHICAGOBULLS',
'SPURS', 'SUNS', 'HOUSTONROCKETS', 'WARRIORS', 'ATLHAWKS', 'MEMGRIZZ', 'BUCKS', 'RAPTORS', 'SACRAMENTOKINGS', 'SIXERS', 'TRAILBLAZERS'}

#coverts to list of strings
def convertType(toConv, makeUpper):

    toConv = toConv.str.replace("'", "")
    toConv = toConv.str.replace("@", " ")
    
    if makeUpper:
        toConv = toConv.str.upper()
          
    toConv = toConv.str.strip("[]")
    toConv = toConv.str.split(',')
    
    count1 = 0
    for x in toConv:
        count2 = 0
        for y in x:
            toConv[count1][count2] = y.strip()
            count2 = count2 + 1
        count1 = count1 + 1
        
    return toConv

def convertTypeTweet(toConv):

    toConv = toConv.str.replace("'", "")
    toConv = toConv.str.replace("@", " ")
          
    toConv = toConv.str.strip("[]")
    toConv = toConv.str.split(',')
    
    count1 = 0
    for x in toConv:
        count2 = 0
        for y in x:
            toConv[count1][count2] = y.strip()
            count2 = count2 + 1
        count1 = count1 + 1
        
    return toConv

# Helper: counts time value is in list of lists
def countContains(toCount, data):
    count = 0
    for x in data:
        if toCount in x:
            count = count + 1
    return count

# Helper: stores value when value in list of list is found
def countLengths(toCount, data1, data2):
    count = []
    for x, y in zip(data1, data2):
        if toCount in x:
            count.append(y)
    return np.mean(count)
    
# convert types  
df = df.drop(['Unnamed: 0'], axis=1)
df = df.drop(['to'], axis=1)
df = df.drop(['from'], axis=1)
df['names'] = convertType(df['names'], False)
df['usernames'] = convertType(df['usernames'], True)
df['dates'] = convertType(df['dates'], True)
df['tweets'] = convertTypeTweet(df['tweets'])

lens = []
for ob in df["names"]:
    lens.append(len(ob))
df["num_parties"] = lens


two_parties = df.loc[df["num_parties"] == 2]
sort = df.sort_values(by='num_parties', ascending=False) 

                                                   head  \
1392  Steven Adams HOOK! He's got 4 early points and...   
1393  Good to see both Rudy and Dieng not hurt on th...   
1394                Overtime?pic.twitter.com/VOTQxxWspR   
1395  .@MNTimberwolves this game...pic.twitter.com/N...   
1396  Would you care for a dance, @SacramentoKings?p...   
1397  Rip City Gameday. #MILvPORpic.twitter.com/3HOe...   
1398  Congrats to the @cavs on a well-deserved serie...   
1399  I did. I've been listening to that since high ...   
1400      Dang, @MNTimberwolves not making this easy...   
1401  #Pelicans head to Sacramento tonight to face t...   
1402  Bulls up 8-4 early on RT @JeffGurt: Joakim Noa...   
1403             20/20 Visionpic.twitter.com/7OrIWuto9f   
1404  .@ATolliver44 for three! More of those, please...   
1405  57 double-doubles on the season for Dre. #Detr...   
1406  Hold up, hold up, hold up. This wasn’t part of...   
1407             . @BenoUdrih1 is cookin! Now w/ 17 pts 

In [4]:
## What is the typical converation size? ##########
df['size'].describe()

count    2982.000000
mean        3.199531
std         2.175947
min         2.000000
25%         2.000000
50%         2.000000
75%         3.000000
max        11.000000
Name: size, dtype: float64

In [28]:
### What teams Tweet the most #######
names = []
participations = []
lengths = []
for name in NBACAPS:
    names.append(name)
    participations.append(countContains(name,df['usernames']))
    lengths.append(countLengths(name,df['usernames'],df['size']))

teams = pd.DataFrame(
        {'name': names,
         'participation': participations,
         'avg_lengths': lengths
        })
    
teams = teams.sort_values(by='participation', ascending=False)
print(teams)
print("Participation " + str(np.mean(teams['participation'])))

#What teams have the longest and shortest conversations
teams = teams.sort_values(by='avg_lengths', ascending=False)
print(teams)

#What teams talk to other teams the most/least? 

def countDuoConvos(name, name2):
    count = 0
    for x in df['usernames']:
        if name in x:
            if name2 in x:
                count = count + 1
    return count
 
tos = []
froms = []
convo_counts = []

for name in NBACAPS:
    for name2 in NBACAPS:
        if name != name2:
            if name < name2:
                tos.append(name)
                froms.append(name2)
                convo_counts.append(countDuoConvos(name, name2))
                
convos = pd.DataFrame(
        {'team_1': tos,
         'team_2': froms,
         'convo_count': convo_counts
        })
 
convos = convos.sort_values(by='convo_count', ascending=False)       
print(convos)
print(np.mean(convos['convo_count']))

    avg_lengths             name  participation
10     2.815327     TRAILBLAZERS            796
6      4.104435         ATLHAWKS            699
29     2.992874  SACRAMENTOKINGS            421
28     3.480000         WARRIORS            400
0      3.423858     TIMBERWOLVES            394
5      4.471850             SUNS            373
20     3.219653       LACLIPPERS            346
19     3.812883           SIXERS            326
7      3.379421         UTAHJAZZ            311
3      3.063433         MEMGRIZZ            268
22     3.926641            BUCKS            259
14     6.059406       DALLASMAVS            202
23     6.244048          HORNETS            168
21     4.283951          NUGGETS            162
2      3.655629      WASHWIZARDS            151
26     3.202797     ORLANDOMAGIC            143
13     2.925373          RAPTORS            134
11     2.632479      PELICANSNBA            117
12     4.513514   DETROITPISTONS            111
1      2.550459     BROOKLYNNETS        

In [9]:
# What percent of each team is sent to other teams

team_name = []
to_name = []
percent = []
for t_index, t_row in teams.iterrows():
    total = t_row['participation']
    name = t_row['name']
    for c_index, c_row in convos.iterrows():
        if (name == c_row['team_1']):
            team_name.append(name)
            to_name.append(c_row['team_2'])
            count = c_row['convo_count']
            if (count == 0):
                percent.append(0)
            else:
                percent.append(c_row['convo_count']/total)
            
convo_percent = pd.DataFrame(
        {'from': team_name,
         'to': to_name,
         'percent': percent
        })
        
convo_percent = convo_percent.sort_values(by='percent', ascending=False)  
hawks = convo_percent.loc[convo_percent["from"] == "ATLHAWKS"]
print(convo_percent)

               from   percent               to
0           HORNETS  0.523810         ATLHAWKS
696      OKCTHUNDER  0.480000     TRAILBLAZERS
29       DALLASMAVS  0.425743         ATLHAWKS
1           HORNETS  0.351190       DALLASMAVS
551            CAVS  0.297030         ATLHAWKS
30       DALLASMAVS  0.292079          HORNETS
377        WARRIORS  0.275000     TRAILBLAZERS
319     WASHWIZARDS  0.271523         ATLHAWKS
58   DETROITPISTONS  0.270270         ATLHAWKS
261        NYKNICKS  0.269231           SIXERS
59   DETROITPISTONS  0.261261     TIMBERWOLVES
841          LAKERS  0.250000     TRAILBLAZERS
232    CHICAGOBULLS  0.250000  SACRAMENTOKINGS
117         CELTICS  0.236842         ATLHAWKS
116         CELTICS  0.236842     TRAILBLAZERS
435        UTAHJAZZ  0.234727     TRAILBLAZERS
638  HOUSTONROCKETS  0.230769     TRAILBLAZERS
87             SUNS  0.227882         ATLHAWKS
464      LACLIPPERS  0.219653     TRAILBLAZERS
493    ORLANDOMAGIC  0.216783         ATLHAWKS
60   DETROITP

In [20]:
# How has the number of conversations changed over time?

years = []
for index, row in df.iterrows():
    year = row['dates'][0][-4:]
    years.append(year)

output = set()
for x in years:
    output.add(x)
output = sorted(list(output))
f_years = []
counts = []

for x in output:
    f_years.append(x)
    counts.append(years.count(x))
    
year_count = pd.DataFrame(
        {'year': f_years,
         'count': counts
        }) 
        
print(year_count)

   count  year
0     10  2012
1     50  2013
2    545  2014
3    890  2015
4    665  2016
5    822  2017


In [55]:
# What was the language of the tweets like/ \sentiment analysis

tweets = df['tweets']
names = df['usernames']
text = []
team = []

for tweet in tweets:
    b = eval(str(tweet))
    
    for c in b:
        text.append(c)
        
for name in names:
    for n in name:
        team.append(n)

neg = []
pos = []
neu = []
comp = []

analyzer = SentimentIntensityAnalyzer()

for t in text:
   vs = analyzer.polarity_scores(t)
   neg.append(vs['neg'])
   pos.append(vs['pos'])
   neu.append(vs['neu'])
   comp.append(vs['compound'])

print(len(text))
print(len(team))
print(len(pos))
print(len(neu))
print(len(neg))
print(len(comp))
all_tweets = pd.DataFrame(
        {'text': text,
         'team': team,
         'neg' : neg,
         'pos' : pos,
         'neu' : neu,
         'comp' : comp
        }) 
      
all_tweets=all_tweets[all_tweets['team'].isin(NBACAPS)]

print("Pos Avg: " + str(all_tweets['pos'].mean()))
print("Pos Max: " + str(all_tweets['pos'].max()))

print("Neg Avg: " + str(all_tweets['neg'].mean()))
print("Neg Max: " + str(all_tweets['neg'].max()))

print("Neu Avg: " + str(all_tweets['neu'].mean()))

print("Comp Avg: " + str(all_tweets['comp'].mean()))

negative = all_tweets[all_tweets['neg'] == 1]
print("\nMost Negative Tweets")
print(negative)

positive = all_tweets[all_tweets['pos'] == 1]
print("\nMost Positive Tweets")
print(positive)

pos_groups = all_tweets.groupby('team')['pos'].mean()
pos_groups = pos_groups.sort_values(ascending=False)
print("\nTeam Pos Avg")
print(pos_groups)

neg_groups = all_tweets.groupby('team')['neg'].mean()
neg_groups = neg_groups.sort_values(ascending=False)
print("\nTeam Neg Avg")
print(neg_groups)

comp_groups = all_tweets.groupby('team')['comp'].mean()
comp_groups = comp_groups.sort_values(ascending=False)
print("\nTeam Comp Avg")
print(comp_groups)

11258
9541


ValueError: arrays must all be same length